In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.0.0'

# 1. 텐서
텐서는 다차원 배열입니다. 넘파이(NumPy) ndarray 객체와 비슷하며, tf.Tensor 객체는 데이터 타입과 크기를 가지고 있습니다. 
또한 tf.Tensor는 GPU 같은 가속기 메모리에 상주할 수 있습니다. 텐서플로는 텐서를 생성하고 이용하는 풍부한 연산 라이브러리(tf.add, tf.matmul, tf.linalg.inv 등.)를 제공합니다. 
이러한 연산은 자동으로 텐서를 파이썬 네이티브(native) 타입으로 변환합니다

In [4]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))

# 연산자 오버로딩(overloading) 또한 지원합니다.
print(tf.square(2) + tf.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)


In [5]:
x = tf.matmul([[1]], [[2, 3]])
print(x)
print(x.shape)
print(x.dtype)

tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>


## 넘파이 배열과 tf.Tensor의 가장 확연한 차이는 다음과 같습니다:

텐서는 가속기 메모리(GPU, TPU와 같은)에서 사용할 수 있습니다.
텐서는 불변성(immutable)을 가집니다.

## 넘파이 호환성

텐서와 넘파이 배열 사이의 변환은 다소 간단합니다.

 - 텐서플로 연산은 자동으로 넘파이 배열을 텐서로 변환합니다.
 - 넘파이 연산은 자동으로 텐서를 넘파이 배열로 변환합니다.
 - 텐서는 .numpy() 메서드(method)를 호출하여 넘파이 배열로 변환할 수 있습니다. 
 
가능한 경우, tf.Tensor와 배열은 메모리 표현을 공유하기 때문에 이러한 변환은 일반적으로 간단(저렴)합니다. 
그러나 tf.Tensor는 GPU 메모리에 저장될 수 있고, 넘파이 배열은 항상 호스트 메모리에 저장되므로, 이러한 변환이 항상 가능한 것은 아닙니다. 
따라서 GPU에서 호스트 메모리로 복사가 필요합니다.

In [7]:
import numpy as np

ndarray = np.ones([3, 3])

print("텐서플로 연산은 자동적으로 넘파이 배열을 텐서로 변환합니다.")
tensor = tf.multiply(ndarray, 42)
print(tensor)

텐서플로 연산은 자동적으로 넘파이 배열을 텐서로 변환합니다.
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)


In [8]:
print("그리고 넘파이 연산은 자동적으로 텐서를 넘파이 배열로 변환합니다.")
print(np.add(tensor, 1))

그리고 넘파이 연산은 자동적으로 텐서를 넘파이 배열로 변환합니다.
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]


In [9]:
print(".numpy() 메서드는 텐서를 넘파이 배열로 변환합니다.")
print(tensor.numpy())

.numpy() 메서드는 텐서를 넘파이 배열로 변환합니다.
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


## GPU 가속

대부분의 텐서플로 연산은 GPU를 사용하여 가속화됩니다. 어떠한 코드를 명시하지 않아도, 텐서플로는 연산을 위해 CPU 또는 GPU를 사용할 것인지를 자동으로 결정합니다. 

필요시 텐서를 CPU와 GPU 메모리 사이에서 복사합니다.
연산에 의해 생성된 텐서는 전형적으로 연산이 실행된 장치의 메모리에 의해 실행됩니다. 

In [10]:
x = tf.random.uniform([3, 3])

print("GPU 사용이 가능한가 : "),
print(tf.test.is_gpu_available())

print("텐서가 GPU #0에 있는가 : "),
print(x.device.endswith('GPU:0'))

GPU 사용이 가능한가 : 
False
텐서가 GPU #0에 있는가 : 
False


In [12]:
import time

def time_matmul(x):
  start = time.time()
  for loop in range(10):
    tf.matmul(x, x)

  result = time.time()-start

  print("10 loops: {:0.2f}ms".format(1000*result))

# CPU에서 강제 실행합니다.
print("On CPU:")
with tf.device("CPU:0"):
  x = tf.random.uniform([1000, 1000])
  assert x.device.endswith("CPU:0")
  time_matmul(x)

On CPU:
10 loops: 154.69ms


In [13]:
# GPU #0가 이용가능시 GPU #0에서 강제 실행합니다.
if tf.test.is_gpu_available():
  print("On GPU:")
  with tf.device("GPU:0"): # Or GPU:1 for the 2nd GPU, GPU:2 for the 3rd etc.
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("GPU:0")
    time_matmul(x)